# Path Finder

## Setup Spark environment

In [1]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-path-finder", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5924,application_1618324153128_5321,pyspark,busy,Link,Link,None,
5978,application_1618324153128_5403,pyspark,idle,Link,Link,None,
5983,application_1618324153128_5409,pyspark,idle,Link,Link,None,
5986,application_1618324153128_5413,pyspark,idle,Link,Link,None,
5994,application_1618324153128_5423,pyspark,idle,Link,Link,None,
6005,application_1618324153128_5438,pyspark,idle,Link,Link,None,
6008,application_1618324153128_5441,pyspark,idle,Link,Link,None,
6009,application_1618324153128_5442,pyspark,idle,Link,Link,None,
6012,application_1618324153128_5445,pyspark,idle,Link,Link,None,
6013,application_1618324153128_5446,pyspark,idle,Link,Link,None,


In [2]:
%%send_to_spark -i username -t str -n username

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6045,application_1618324153128_5484,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Import data from HDFS

In [4]:
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
stops = spark.read.orc('/data/sbb/orc/geostops')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
walking_edges = spark.read.parquet('/user/%s/final/parquet/walking_edges' % username)
transport_edges = spark.read.parquet('/user/%s/final/parquet/transport_edges' % username)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
transport_edges.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id', 'weekday']

## Shortest Path

In [7]:
walking_edges = walking_edges.withColumn('value', F.struct(F.col('end_vertex'), F.col('duration')))\
                             .groupBy('start_vertex').agg({'value': 'collect_set'})\
                             .toPandas()\
                             .set_index('start_vertex')\
                             .to_dict()['collect_set(value)']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
edges = transport_edges.filter('weekday == "monday"')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, walking_edges):

    def edge_valid(u, v, time, dur, route_id): 
        next_edges = all_roads.get(v, [])
        
        # Can we reach the destination ?
        if v == end_stop and time + dur <= end_time: 
             return (time, v, route_id, dur)
        
        # We don't add edges starting at the destination
        if u != end_stop:
            for edge in next_edges[::-1]: # Traverse the list in reverse order because it's sorted by descending starting time
                time_v, next_v, route_id_v, dur_v = edge
                if (route_id == route_id_v and time + dur <= time_v) or (time + dur + transfer_time <= time_v): 
                    return (time, v, route_id, dur)            
            
            # Can we walk from v instead?
            for end_vertex, walking_duration in walking_edges.get(v, []):
                if end_vertex != u: # do not loop back to u
                    for next_next_edge in all_roads.get(end_vertex, []):
                        time_next = next_next_edge[0]
                        if time + dur <= time_next - walking_duration - transfer_time : # can we make in time to v in order to walk
                            return (time, v, route_id, dur)
                
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10
    
    all_roads = {}
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
        
    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas().sort_values(by=['start_time'], ascending=False).to_numpy()

    for row in edges:
        start_vertex, end_vertex, start, duration, route_id = row[:5]
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            if start_vertex in all_roads:    
                all_roads[start_vertex].append(update)
            else:
                all_roads[start_vertex] = [update]
                        
    return all_roads

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
import time
start = time.time()
all_roads = latest_departure_paths(edges, '8591221', '20:00', walking_edges)
end = time.time() 
print(end - start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.64788603783

In [11]:
stop_id_to_stop = stops.select('stop_id', 'stop_name')
stop_id_to_stop_dict = stop_id_to_stop.toPandas().set_index('stop_id').to_dict()['stop_name']

route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')
route_id_to_route_name_dict = route_id_to_route_name.toPandas().set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['Walking'] = 'Walk'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
# Journey reconstruction

from itertools import groupby
import numpy as np

def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)

def select_next(edge, all_roads, walking_edges, transfer_time=2):
    
    prev_time, node, prev_route_id, prev_duration = edge[:4]
    
    next_edges = all_roads.get(node, [])
    
    if prev_route_id == 'Walking':
        return [next_edges[edge[-1]]]
    else:
        possible_edges = []
        next_edges = next_edges +  get_walking_edges(all_roads, node, walking_edges)
        for edge_n in next_edges:
            t, v, route_id, dur = edge_n[:4]
            if prev_route_id != route_id and prev_time + prev_duration + transfer_time <= t:
                possible_edges.append(edge_n)
            elif prev_route_id == route_id and prev_time + prev_duration <= t:
                possible_edges.append(edge_n)

        return possible_edges 
    
def iterate_all_edges(start_edges, end_stop, all_roads, walking_edges, cur_list):
    for edge in start_edges:
        t, v, route_id, dur = edge[:4]
        if v == end_stop: # have we reached the destination?
            yield cur_list + [edge]
        else:
            next_edges = select_next(edge,  all_roads, walking_edges)
            for journey in iterate_all_edges(next_edges, end_stop, all_roads, walking_edges, cur_list + [edge]):
                yield journey

def get_journey_attributes(journey):
    walking_distance = sum([t[-1] for t in journey if t[-1] == 'Walking'])
    transfers = len(set([t[2] for t in journey if t[2] != 'Walking']))
    return [transfers, walking_distance]
    
def get_walking_edges(all_roads, node, walking_edges, transfer_time=2):
    # Compute viable walks starting at node u:
    possible_walking_edges = []
    for end_vertex, duration in walking_edges.get(node, []):
        for index, edge in enumerate(all_roads.get(end_vertex, [])):
            time_next, node_next = edge[:2]
            if node_next != node:
                possible_walking_edges.append((time_next - duration - transfer_time, end_vertex, 'Walking', duration, index))
    return possible_walking_edges
    
def generate_best_paths(all_roads, start_stop, end_stop, walking_edges):
    # add possible paths that start by walking
    start_edges = all_roads.get(start_stop, []) + get_walking_edges(all_roads, start_stop, walking_edges)
    if not start_edges: # not paths were found
        yield []
    else:
        # sort edges and group by start_time
        start_edges = sorted(start_edges, key=lambda x:x[0], reverse=True)
        start_edges = groupby(start_edges, key=lambda x:x[0])
        
        for start_time , edges in start_edges:
            # find all possible journeys that start at time start_time
            all_journeys = list(iterate_all_edges(list(edges), end_stop, all_roads, walking_edges, []))
            journey_attribs = np.array([get_journey_attributes(journey) for journey in all_journeys])
            sorted_indices = np.lexsort(journey_attribs.T)
            for i in sorted_indices:
                yield all_journeys[i]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
start_stop = all_roads.keys()[29]
end_stop = '8591221'
iterator = generate_best_paths(all_roads, start_stop, end_stop, walking_edges)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
for l in all_roads.values():
    for i in l:
        try:
            assert len(i) == 4
        except AssertionError:
            print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
journey = next(iterator)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
journey

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1134, u'8590279', u'26-303-j19-1', 2), (1136, u'8591891', u'26-303-j19-1', 1), (1137, u'8590270', u'26-303-j19-1', 2), (1139, u'8502508', u'26-303-j19-1', 1), (1140, u'8590268', u'26-303-j19-1', 0), (1140, u'8590218', u'26-303-j19-1', 2), (1142, u'8590222', u'26-303-j19-1', 1), (1143, u'8590535', u'26-303-j19-1', 2), (1145, u'8587020:0:A', u'26-303-j19-1', 2), (1152, u'8503508:0:4', 'Walking', 1, 0), (1155, u'8503512:0:2', u'26-11-j19-1', 2), (1157, u'8503509:0:4', u'26-11-j19-1', 2), (1159, u'8503001:0:4', u'26-11-j19-1', 4), (1164, u'8503020:0:1', u'26-11-j19-1', 2), (1166, u'8503000:0:43/44', u'26-11-j19-1', 4), (1171, u'8503003:0:3', u'26-11-j19-1', 3), (1179, u'8591375', 'Walking', 6, 1), (1187, u'8530813', u'26-31-j19-1', 1), (1188, u'8591364', u'26-31-j19-1', 2), (1190, u'8530812', u'26-31-j19-1', 1), (1192, u'8591137', u'26-31-j19-1', 1), (1193, u'8591233', u'26-31-j19-1', 1), (1194, u'8591221', u'26-31-j19-1', 2)]

In [77]:
def journey_to_segments (journey, start_stop):
    segments = [(route_id, list(edges)) for (route_id, edges) in  groupby(journey, lambda x:x[2])]

    segments = [(i + 1, edges[0][1], edges[-1][1], route_id, edges[0][0], edges[-1][0] + edges[-1][3] - edges[0][0], [t[1] for t in edges]) \
                for i, (route_id, edges) in enumerate(segments)]
    
    segments[0] = (segments[0][0], start_stop, segments[0][2], segments[0][3], segments[0][4], segments[0][5], [start_stop] + segments[0][6])
    
    return segments

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
journey_to_segments(journey, start_stop)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, u'8590397', u'8587020:0:A', u'26-303-j19-1', 1134, 13, [u'8590397', u'8590279', u'8591891', u'8590270', u'8502508', u'8590268', u'8590218', u'8590222', u'8590535', u'8587020:0:A']), (2, u'8503508:0:4', u'8503508:0:4', 'Walking', 1152, 1, [u'8503508:0:4']), (3, u'8503512:0:2', u'8503003:0:3', u'26-11-j19-1', 1155, 19, [u'8503512:0:2', u'8503509:0:4', u'8503001:0:4', u'8503020:0:1', u'8503000:0:43/44', u'8503003:0:3']), (4, u'8591375', u'8591375', 'Walking', 1180, 6, [u'8591375']), (5, u'8530813', u'8591221', u'26-31-j19-1', 1188, 8, [u'8530813', u'8591364', u'8530812', u'8591137', u'8591233', u'8591221'])]

In [154]:
def journey_to_df(journey_list):
    arr = []
    journey_number = 0
    for journey in journey_list:
        journey_number += 1
        arr = arr + [(journey_number,) + seg for seg in journey]
    
    schema = StructType([
        StructField("journey_number", IntegerType(), True),
        StructField("segment_number", IntegerType(), True),
        StructField("start_vertex", StringType(), True),
        StructField("end_vertex", StringType(), True),
        StructField("route_id", StringType(), True),
        StructField("departure_time", StringType(), True),
        StructField("duration", StringType(), True),
        StructField("stop_seq", ArrayType(StringType()), True)
    ])
    
    journey_df = spark.createDataFrame(arr, schema)
    
    return journey_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
journey_df = journey_to_df([journey_to_segments(journey, '8530822')])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min)\n| %s\n|\n%s : %s\n" % (minutes_to_timestamp(time), 
                                                                             stop_id_to_stop_dict[start],
                                                                             route_id_to_route_name_dict[route_id],
                                                                             minutes_to_timestamp(duration),
                                                                             ' -> '.join([stop_id_to_stop_dict[s] for s in stops]) if route_id != 'Walking' else '',
                                                                             minutes_to_timestamp(time + duration),
                                                                             stop_id_to_stop_dict[end]) for _, start, end, route_id, time, duration, stops in journey])
    return journey_str

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Send to local

Send transfer dataframe to local

In [ ]:
%%spark -o next_transfer_df -n -1

In [ ]:
%%local
next_transfer_df.head()

Reconstruct a dict for transfers to keep old code

In [ ]:
%%local
next_transfer = {e[0]: tuple(e[1:]) for e in next_transfer_df.to_numpy()}

Send route id to route name mapping to local

In [ ]:
%%spark -o route_id_to_route_name -n -1
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')

In [ ]:
%%local
route_id_to_route_name_dict = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'walk'

Send stop id to stop name mapping to local

In [ ]:
%%spark -o stop_id_to_stop -n -1
stop_id_to_stop = stops.select('stop_id', 'stop_name')

In [ ]:
%%local
stop_id_to_stop_dict = stop_id_to_stop.set_index('stop_id').to_dict()['stop_name']

## Reconstruct path and pretty print

In [ ]:
%%local
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
    
def reconstruct_journey(next_transfer, start_stop):
    time, next_stop, route_id, duration, stop_seq = next_transfer[start_stop]
    journey = []
    while next_stop != start_stop:
        journey.append(
            (minutes_to_timestamp(time),
             stop_id_to_stop_dict[start_stop], 
             route_id_to_route_name_dict[route_id],
             duration,
             'Ride %d stops' % (len(stop_seq)-1),
             minutes_to_timestamp(time + duration),
             stop_id_to_stop_dict[next_stop])
        )
        
        start_stop = next_stop
        time, next_stop, route_id, duration, stop_seq = next_transfer[next_stop]
    return journey


def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min) %s\n|\n%s : %s\n" % segment for segment in journey])
    return journey_str

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590883'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591353'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590788'))) # not in sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591368'))) # Correct